In [92]:
import awswrangler as wr
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
import os
import sys

In [16]:
repo_name = "mercury-pipeline-metaapi-sam"
sys.path.append(os.getcwd().split(repo_name)[0] + repo_name + "/lambda") 

In [17]:
import app

In [56]:
import random
import string

In [101]:
def apply_id(
    df: pd.DataFrame,
    label: str
) -> pd.DataFrame:
    mask = df[label] != 0.0
    df[label] = np.nan
    df.loc[mask, label] = "".join(random.choices(string.ascii_uppercase, k=20))
    return df

def ema(
    df: pd.DataFrame
) -> pd.DataFrame:
    
    for av in [1, 10, 21, 50]:
        df[f"ema_{av}"] = df["ask.close"].ewm(span=av, adjust=False).mean()
        
    # up and downtrends
    df["ema_up"] = (df["ema_10"] > df["ema_21"]) & (df["ema_10"] > df["ema_50"])
    df["ema_dn"] = (df["ema_10"] < df["ema_21"]) & (df["ema_10"] < df["ema_50"])
    
    df["ema_up_start"] = df["ema_up"] > df["ema_up"].shift()
    df["ema_dn_start"] = df["ema_dn"] > df["ema_dn"].shift()
    df["ema_up_stop"] = df["ema_up"] < df["ema_up"].shift()
    df["ema_dn_stop"] = df["ema_dn"] < df["ema_dn"].shift()
    
    df["ema_up"] = df["ema_up"] | df["ema_up_stop"]
    df["ema_dn"] = df["ema_dn"] | df["ema_dn_stop"]

    # Get a unique index for each individual trend
    df["ema_up_i"] = df["ema_up_start"].cumsum() * df["ema_up"]
    df["ema_dn_i"] = df["ema_dn_start"].cumsum() * df["ema_dn"]

    df = df.groupby("ema_up_i").apply(apply_id, label="ema_up_i")
    df = df.groupby("ema_dn_i").apply(apply_id, label="ema_dn_i")

    return df

In [164]:
def trend_properties_micro(
    df: pd.DataFrame,
    trend_type: str
) -> pd.DataFrame:
    # Make sure that points are in time order (they should be anyway but hey ho).
    df = (
        df.sort_values(by="timestamp_utc.open", ascending=True)
        .reset_index(drop=True)
        .reset_index(drop=False)
        .rename(columns={"index":f"trend_{trend_type}_idx"})
        .drop(columns=[f"ema_{trend_type}_i"])
    )
    
    df[f"ema_{trend_type}_peak"] = {
        "up": df["bid.high"].idxmax(),
        "dn": df["ask.low"].idxmin()
    }[trend_type]
    df[f"ema_{trend_type}_trough"] = {
        "up": df["bid.low"].idxmin(),
        "dn": df["ask.high"].idxmax()
    }[trend_type]

    return df[[
        "timestamp_utc.open",
        f"ema_{trend_type}_peak",
        f"ema_{trend_type}_trough",
    ]]

In [166]:
def trend_micro(
    df: pd.DataFrame
) -> pd.DataFrame:
    
    n_start = len(df)
    
    mask = df["ema_up"] == 1
    if len(df[mask]) > 0:
        dfi = (
            df[mask].groupby("ema_up_i")
            .apply(trend_properties_micro, trend_type="up")
            .reset_index()
            .drop(columns=["level_1"])
        )
        df = pd.merge(
            df,
            dfi,
            how="left",
            on=["timestamp_utc.open", "ema_up_i"]
        )
        
    mask = df["ema_dn"] == 1
    if len(df[mask]) > 0:
        dfi = (
            df[mask].groupby("ema_dn_i")
            .apply(trend_properties_micro, trend_type="dn")
            .reset_index()
            .drop(columns=["level_1"])
        )
        df = pd.merge(
            df,
            dfi,
            how="left",
            on=["timestamp_utc.open", "ema_dn_i"]
        )
        
    df = df.sort_values(by="timestamp_utc.open")
    assert len(df) == n_start
    return df

In [146]:
pd.set_option('display.max_columns', None)

In [147]:
path = "s3://datalake.dgriffiths.io/projects/mercury/data/clean/candles/ffill/symbol=EURUSD/frequency=1T/year=2022/month=07/date=20220714/"

In [148]:
df = wr.s3.read_parquet(path, dataset=True)

In [149]:
df = ema(df)

In [150]:
df.head(3)

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close,symbol,frequency,year,month,date,hour,ema_1,ema_10,ema_21,ema_50,ema_up,ema_dn,ema_up_start,ema_dn_start,ema_up_stop,ema_dn_stop,ema_up_i,ema_dn_i
0,2022-07-14 00:00:00,1.00352,1.00342,1.00324,1.00364,85,1.00331,1.00323,1.00305,1.00345,hi,hi,2022-07-14 00:01:00,2022-07-14 01:00:00,2022-07-14 01:01:00,EURUSD,1T,2022,07,20220714,00,1.00342,1.003420,1.003420,1.003420,False,False,False,False,False,False,NaN,NaN
1,2022-07-14 00:01:00,1.00343,1.00373,1.00323,1.00376,88,1.00323,1.00354,1.00304,1.00357,lo,lo,2022-07-14 00:02:00,2022-07-14 01:01:00,2022-07-14 01:02:00,EURUSD,1T,2022,07,20220714,00,1.00373,1.003476,1.003448,1.003432,True,False,True,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN
2,2022-07-14 00:02:00,1.00376,1.00360,1.00360,1.00391,67,1.00357,1.00341,1.00341,1.00372,hi,hi,2022-07-14 00:03:00,2022-07-14 01:02:00,2022-07-14 01:03:00,EURUSD,1T,2022,07,20220714,00,1.00360,1.003499,1.003462,1.003439,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN


In [151]:
df_trend = df[df["ema_up_i"] == df["ema_up_i"].dropna().unique()[0]]

In [152]:
df_trend = trend_properties_micro(
    df_trend,
    "up"
)

In [153]:
df_trend

,trend_up_idx,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close,symbol,frequency,year,month,date,hour,ema_1,ema_10,ema_21,ema_50,ema_up,ema_dn,ema_up_start,ema_dn_start,ema_up_stop,ema_dn_stop,ema_up_i,ema_dn_i,i_peak,i_trough
0,0,2022-07-14 00:01:00,1.00343,1.00373,1.00323,1.00376,88,1.00323,1.00354,1.00304,1.00357,lo,lo,2022-07-14 00:02:00,2022-07-14 01:01:00,2022-07-14 01:02:00,EURUSD,1T,2022,07,20220714,00,1.00373,1.003476,1.003448,1.003432,True,False,True,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
1,1,2022-07-14 00:02:00,1.00376,1.00360,1.00360,1.00391,67,1.00357,1.00341,1.00341,1.00372,hi,hi,2022-07-14 00:03:00,2022-07-14 01:02:00,2022-07-14 01:03:00,EURUSD,1T,2022,07,20220714,00,1.00360,1.003499,1.003462,1.003439,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
2,2,2022-07-14 00:03:00,1.00362,1.00374,1.00362,1.00386,72,1.00341,1.00355,1.00341,1.00367,lo,lo,2022-07-14 00:04:00,2022-07-14 01:03:00,2022-07-14 01:04:00,EURUSD,1T,2022,07,20220714,00,1.00374,1.003543,1.003487,1.003451,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
3,3,2022-07-14 00:04:00,1.00378,1.00394,1.00370,1.00402,54,1.00355,1.00375,1.00351,1.00383,lo,lo,2022-07-14 00:05:00,2022-07-14 01:04:00,2022-07-14 01:05:00,EURUSD,1T,2022,07,20220714,00,1.00394,1.003615,1.003528,1.003470,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
4,4,2022-07-14 00:05:00,1.00396,1.00404,1.00391,1.00406,39,1.00375,1.00385,1.00372,1.00387,hi,hi,2022-07-14 00:06:00,2022-07-14 01:05:00,2022-07-14 01:06:00,EURUSD,1T,2022,07,20220714,00,1.00404,1.003692,1.003575,1.003492,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
5,5,2022-07-14 00:06:00,1.00406,1.00411,1.00405,1.00416,46,1.00385,1.00392,1.00385,1.00397,lo,lo,2022-07-14 00:07:00,2022-07-14 01:06:00,2022-07-14 01:07:00,EURUSD,1T,2022,07,20220714,00,1.00411,1.003768,1.003624,1.003516,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
6,6,2022-07-14 00:07:00,1.00412,1.00416,1.00402,1.00417,30,1.00392,1.00397,1.00383,1.00398,lo,lo,2022-07-14 00:08:00,2022-07-14 01:07:00,2022-07-14 01:08:00,EURUSD,1T,2022,07,20220714,00,1.00416,1.003839,1.003672,1.003542,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
7,7,2022-07-14 00:08:00,1.00417,1.00386,1.00385,1.00417,66,1.00397,1.00367,1.00366,1.00398,hi,hi,2022-07-14 00:09:00,2022-07-14 01:08:00,2022-07-14 01:09:00,EURUSD,1T,2022,07,20220714,00,1.00386,1.003843,1.003689,1.003554,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
8,8,2022-07-14 00:09:00,1.00387,1.00426,1.00387,1.00426,35,1.00367,1.00407,1.00367,1.00407,lo,lo,2022-07-14 00:10:00,2022-07-14 01:09:00,2022-07-14 01:10:00,EURUSD,1T,2022,07,20220714,00,1.00426,1.003919,1.003741,1.003582,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0
9,9,2022-07-14 00:10:00,1.00427,1.00435,1.00417,1.00436,63,1.00407,1.00416,1.00398,1.00417,hi,hi,2022-07-14 00:11:00,2022-07-14 01:10:00,2022-07-14 01:11:00,EURUSD,1T,2022,07,20220714,00,1.00435,1.003997,1.003797,1.003612,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10,0


In [167]:
trend_micro(df)

,timestamp_utc.open,ask.open,ask.close,ask.low,ask.high,n_ticks,bid.open,bid.close,bid.low,bid.high,ask.idx_hi_lo,bid.idx_hi_lo,timestamp_utc.close,timestamp_uk.open,timestamp_uk.close,symbol,frequency,year,month,date,hour,ema_1,ema_10,ema_21,ema_50,ema_up,ema_dn,ema_up_start,ema_dn_start,ema_up_stop,ema_dn_stop,ema_up_i,ema_dn_i,ema_up_peak,ema_up_trough,ema_dn_peak,ema_dn_trough
0,2022-07-14 00:00:00,1.00352,1.00342,1.00324,1.00364,85,1.00331,1.00323,1.00305,1.00345,hi,hi,2022-07-14 00:01:00,2022-07-14 01:00:00,2022-07-14 01:01:00,EURUSD,1T,2022,07,20220714,00,1.00342,1.003420,1.003420,1.003420,False,False,False,False,False,False,NaN,NaN,NaN,NaN,NaN,NaN
1,2022-07-14 00:01:00,1.00343,1.00373,1.00323,1.00376,88,1.00323,1.00354,1.00304,1.00357,lo,lo,2022-07-14 00:02:00,2022-07-14 01:01:00,2022-07-14 01:02:00,EURUSD,1T,2022,07,20220714,00,1.00373,1.003476,1.003448,1.003432,True,False,True,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10.0,0.0,NaN,NaN
2,2022-07-14 00:02:00,1.00376,1.00360,1.00360,1.00391,67,1.00357,1.00341,1.00341,1.00372,hi,hi,2022-07-14 00:03:00,2022-07-14 01:02:00,2022-07-14 01:03:00,EURUSD,1T,2022,07,20220714,00,1.00360,1.003499,1.003462,1.003439,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10.0,0.0,NaN,NaN
3,2022-07-14 00:03:00,1.00362,1.00374,1.00362,1.00386,72,1.00341,1.00355,1.00341,1.00367,lo,lo,2022-07-14 00:04:00,2022-07-14 01:03:00,2022-07-14 01:04:00,EURUSD,1T,2022,07,20220714,00,1.00374,1.003543,1.003487,1.003451,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10.0,0.0,NaN,NaN
4,2022-07-14 00:04:00,1.00378,1.00394,1.00370,1.00402,54,1.00355,1.00375,1.00351,1.00383,lo,lo,2022-07-14 00:05:00,2022-07-14 01:04:00,2022-07-14 01:05:00,EURUSD,1T,2022,07,20220714,00,1.00394,1.003615,1.003528,1.003470,True,False,False,False,False,False,TMINIOHZBQPDQXMLQOXN,NaN,10.0,0.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1435,2022-07-14 23:55:00,1.00258,1.00253,1.00247,1.00258,36,1.00235,1.00230,1.00224,1.00235,hi,hi,2022-07-14 23:56:00,2022-07-15 00:55:00,2022-07-15 00:56:00,EURUSD,1T,2022,07,20220714,23,1.00253,1.002507,1.002402,1.002230,True,False,False,False,False,False,TFCTRPTHWHVGHWVFNBYG,NaN,28.0,0.0,NaN,NaN
1436,2022-07-14 23:56:00,1.00254,1.00254,1.00253,1.00254,11,1.00230,1.00231,1.00230,1.00231,hi,lo,2022-07-14 23:57:00,2022-07-15 00:56:00,2022-07-15 00:57:00,EURUSD,1T,2022,07,20220714,23,1.00254,1.002513,1.002414,1.002242,True,False,False,False,False,False,TFCTRPTHWHVGHWVFNBYG,NaN,28.0,0.0,NaN,NaN
1437,2022-07-14 23:57:00,1.00255,1.00265,1.00253,1.00265,22,1.00231,1.00242,1.00230,1.00242,lo,lo,2022-07-14 23:58:00,2022-07-15 00:57:00,2022-07-15 00:58:00,EURUSD,1T,2022,07,20220714,23,1.00265,1.002538,1.002436,1.002258,True,False,False,False,False,False,TFCTRPTHWHVGHWVFNBYG,NaN,28.0,0.0,NaN,NaN
1438,2022-07-14 23:58:00,1.00264,1.00265,1.00264,1.00265,20,1.00242,1.00242,1.00241,1.00242,lo,hi,2022-07-14 23:59:00,2022-07-15 00:58:00,2022-07-15 00:59:00,EURUSD,1T,2022,07,20220714,23,1.00265,1.002558,1.002455,1.002274,True,False,False,False,False,False,TFCTRPTHWHVGHWVFNBYG,NaN,28.0,0.0,NaN,NaN
